In [1]:
import cv2
import numpy as np

In [2]:
# 가우시안 필터 -> 노이즈 제거
def del_noise(input_mat):
    ret_mat = cv2.GaussianBlur(input_mat, (3,3), 0)

    return ret_mat

In [3]:
# grayscale로 변환, 이진화

def make_gray_and_binary(input_mat):
    gray_mat = cv2.cvtColor(input_mat, cv2.COLOR_BGR2GRAY)
    boundary_value, ret_mat = cv2.threshold(gray_mat, 127, 255, cv2.THRESH_BINARY)
    
    return ret_mat

In [4]:
# 모폴로지(열림)

def morphologing(input_mat):
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    ret_mat = cv2.morphologyEx(input_mat, cv2.MORPH_OPEN, k)

    return ret_mat

In [5]:
# 엣지 검출(라플라시안 필터 or 캐니)

def detect_edge(input_mat, lapla_or_canny):
    if(lapla_or_canny == 0):
        ret_mat = cv2.Laplacian(input_mat, -1)
    elif(lapla_or_canny == 1):
        ret_mat = cv2.Canny(input_mat, 100, 200)
    
    return ret_mat

In [6]:
img = cv2.imread('Dog_11.jpg')
h, w = img.shape[:2]
print(h, w)

224 224


In [7]:
rc = cv2.selectROI(img)
mask = np.zeros(img.shape[:2], dtype= np.uint8)
bgd = np.zeros((1, 65), np.float64)
fgd = np.zeros((1, 65), np.float64)
cv2.grabCut(img, mask, rc, None, None, 5, cv2.GC_INIT_WITH_RECT)

cv2.destroyAllWindows()

mask2 = np.where((mask == 0) | (mask == 2), 0, 1).astype('uint8')

dst = img*mask2[:, :, np.newaxis]

In [8]:
cv2.imshow('dst', dst)
# cv2.imshow('bdg', bgd)
# cv2.imshow('fgd', fgd)
cv2.waitKey()
cv2.destroyAllWindows()

In [9]:
# img_gr = make_gray_and_binary(img)
# cv2.imshow('gray', img_gr)

# contour, hr = cv2.findContours(img_gr, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# x,y,w,h = cv2.boundingRect(contour[0])

# img = cv2.imread('Dog_11.jpg')
# cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255))
# cv2.imshow('img + contour', img)
# cv2.waitKey()
# cv2.destroyAllWindows()

In [10]:
# img = cv2.imread('Dog_11.jpg', cv2.IMREAD_GRAYSCALE)
# img = detect_edge(img, 1)
# cv2.imshow('edge', img)

# contour, hr = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# rotateRect = cv2.minAreaRect(contour)

# vertex = cv2.boxPoints(rotateRect)

# img = cv2.imread('Dog_11.jpg')
# # cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), thickness= 5)
# cv2.drawContours(img, contour, -1, (255, 0, 255), 2)
# cv2.imshow('img + contour', img)
# cv2.waitKey()
# cv2.destroyAllWindows()

# print(x, y, x+w, y+h)

In [17]:
img = cv2.imread('Dog_11.jpg')

data = img.reshape((-1, 3)).astype(np.float32)

criteria = (cv2.TERM_CRITERIA_EPS|cv2.TERM_CRITERIA_MAX_ITER, 10, 5.0)

for K in range(1, 10):
    print('K:', K)
    ret, label, center = cv2.kmeans(data, K, None, criteria, 4, cv2.KMEANS_RANDOM_CENTERS)

    center = np.uint8(center)

    dst = center[label.flatten()]

    dst = dst.reshape((img.shape))

    cv2.imshow('src', img)
    cv2.imshow('dst', dst)
    cv2.waitKey()

cv2.destroyAllWindows()

K: 1
K: 2
K: 3
K: 4
K: 5
K: 6
K: 7
K: 8
K: 9
